In [ ]:
import os

from pathlib import Path

import pandas as pd
import polars as pl


import lightgbm as lgb

import warnings

warnings.filterwarnings('ignore')

#Logging
import logging

# ロガーの取得
logger = logging.getLogger(__name__)
logger.setLevel(logging.DEBUG)
file_handler = logging.FileHandler('logs.log')

# # ログのフォーマットを設定
formatter = logging.Formatter('[%(asctime)s][%(levelname)s] %(message)s')
file_handler.setFormatter(formatter)
logger.addHandler(file_handler)

# Notebook上にもログを表示するためのハンドラ
stream_handler = logging.StreamHandler()
stream_handler.setFormatter(formatter)
logger.addHandler(stream_handler)

pd.set_option('display.max_columns', 1000)

# Inference

In [41]:
articles = pl.read_parquet('/home/data/ebnerd_testset/articles.parquet')

lgb_model_part1 = lgb.Booster(model_file='lgb_model_20p_1.txt')
lgb_model_part2 = lgb.Booster(model_file='lgb_model_20p_2.txt')
lgb_model_part3 = lgb.Booster(model_file='lgb_model_20p_3.txt')
lgb_model_part4 = lgb.Booster(model_file='lgb_model_20p_4.txt')
lgb_model_part5 = lgb.Booster(model_file='lgb_model_20p_5.txt')
lgb_model_part6 = lgb.Booster(model_file='lgb_model_20p_6.txt')
lgb_model_part7 = lgb.Booster(model_file='lgb_model_20p_7.txt')
lgb_model_part8 = lgb.Booster(model_file='lgb_model_20p_8.txt')

feature_cols = lgb_model_part1.feature_name()

In [ ]:
sub_df_list = []
sub_df_list_org = []

for i in range(100):
    logger.info(f'no : {i}')
    
    if f"sub_df_org_chunk{i}.parquet" in os.listdir("./test_predictions/sub_df_orgs/"):
        continue
    
    pl.DataFrame().write_parquet(f"./test_predictions/sub_df_orgs/sub_df_org_chunk{i}.parquet")
    
    pl.DataFrame()
    
    test_df = pl.read_parquet(f"./test_large_chunks/test_df_chunk{i}.parquet")
    test_df = test_df.join(articles.select(["article_id", "category"]), how="left", on="article_id")
    X_test = test_df.select(feature_cols).to_pandas()
    
    X_test["category"] = X_test["category"].astype("category")
    
    logger.info(f'starting prediction')
    lgb_pred_part1 = lgb_model_part1.predict(X_test)
    lgb_pred_part2 = lgb_model_part2.predict(X_test)
    lgb_pred_part3 = lgb_model_part3.predict(X_test)
    lgb_pred_part4 = lgb_model_part4.predict(X_test)
    lgb_pred_part5 = lgb_model_part5.predict(X_test)
    lgb_pred_part6 = lgb_model_part6.predict(X_test)
    lgb_pred_part7 = lgb_model_part7.predict(X_test)
    lgb_pred_part8 = lgb_model_part8.predict(X_test)
    
    logger.info(f'organizing results')
    sub_df_org = pl.DataFrame(
        {
            'impression_id': test_df['impression_id'],
            'article_id': test_df['article_id'],
            'user_id': test_df['user_id'],
            'y_pred1': lgb_pred_part1,
            'y_pred2': lgb_pred_part2,
            'y_pred3': lgb_pred_part3,
            'y_pred4': lgb_pred_part4,
            'y_pred5': lgb_pred_part5,
            'y_pred6': lgb_pred_part6,
            'y_pred7': lgb_pred_part7,
            'y_pred8': lgb_pred_part8,
        }
    )
    
    sub_df = sub_df_org.with_columns([
            pl.col('y_pred1').rank().over('impression_id').alias('y_pred1'),
            pl.col('y_pred2').rank().over('impression_id').alias('y_pred2'),
            pl.col('y_pred3').rank().over('impression_id').alias('y_pred3'),
            pl.col('y_pred4').rank().over('impression_id').alias('y_pred4'),
            pl.col('y_pred5').rank().over('impression_id').alias('y_pred5'),
            pl.col('y_pred6').rank().over('impression_id').alias('y_pred6'),
            pl.col('y_pred7').rank().over('impression_id').alias('y_pred7'),
            pl.col('y_pred8').rank().over('impression_id').alias('y_pred8'),
    ]).with_columns([
        (
                    pl.col('y_pred1')*0.125 + \
                    pl.col('y_pred2')*0.125 + \
                    pl.col('y_pred3')*0.125 + \
                    pl.col('y_pred4')*0.125 + \
                    pl.col('y_pred5')*0.125 + \
                    pl.col('y_pred6')*0.125 + \
                    pl.col('y_pred7')*0.125 + \
                    pl.col('y_pred8')*0.125
        ).alias('pred')
    ])
    
    #impression_idごとにpredの順位をつける
    sub_df = sub_df.groupby(['impression_id','user_id']).agg(
        pl.col('pred').rank(method = 'ordinal',descending = True).alias('prediction_scores')
    )
    
    sub_df_list.append(sub_df)
    sub_df_list_org.append(sub_df_org)
    
    os.remove(f"./test_predictions/sub_df_orgs/sub_df_org_chunk{i}.parquet")
    
    sub_df_org.write_parquet(f"./test_predictions/sub_df_orgs/sub_df_org_chunk{i}.parquet")
    sub_df.write_parquet(f"./test_predictions/sub_dfs/sub_df_chunk{i}.parquet")

[2024-06-20 01:39:12,955][INFO] no : 0
[2024-06-20 01:39:16,673][INFO] starting prediction
[2024-06-20 01:59:39,166][INFO] organizing results
[2024-06-20 01:59:43,068][INFO] no : 1
[2024-06-20 01:59:43,072][INFO] no : 2
[2024-06-20 01:59:43,074][INFO] no : 3
[2024-06-20 01:59:43,077][INFO] no : 4
[2024-06-20 01:59:43,078][INFO] no : 5
[2024-06-20 01:59:43,081][INFO] no : 6
[2024-06-20 01:59:43,083][INFO] no : 7
[2024-06-20 01:59:43,084][INFO] no : 8
[2024-06-20 01:59:49,765][INFO] starting prediction
[2024-06-20 02:23:02,222][INFO] organizing results
[2024-06-20 02:23:06,556][INFO] no : 9
[2024-06-20 02:23:06,562][INFO] no : 10
[2024-06-20 02:23:06,565][INFO] no : 11
[2024-06-20 02:23:06,568][INFO] no : 12
[2024-06-20 02:23:06,570][INFO] no : 13
[2024-06-20 02:23:06,572][INFO] no : 14
[2024-06-20 02:23:06,574][INFO] no : 15
[2024-06-20 02:23:12,515][INFO] starting prediction
[2024-06-20 02:44:27,250][INFO] organizing results
[2024-06-20 02:44:30,684][INFO] no : 16
[2024-06-20 02:44:30,